In [1]:
from yelpapi import YelpAPI
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import time
import re

API_KEY = '3HutAvbl1TRuIXG0mjgNjTGPuLy_ZhoON7UEYzJqG6o2D6gPkIVZr1RT4x3TBpR3iXtJ9lBnis50H7MXRp0uSisapP-8Tup98qEipjCL6AjzLv2jHM6UhMXZN8IBYnYx'
yelp_api = YelpAPI(API_KEY)

In [2]:
# STEP 1: create list of neighborhoods for each midwest city
cities = ['Chicago, Illinois',
 'Columbus, Ohio', 
 'Indianapolis, Indiana',
 'Detroit, Michigan',
 'Milwaukee, Wisconsin',
 'Kansas City, Missouri',
 'St. Louis, Missouri',
 'Minneapolis, Minnesota',
 'Saint Paul, Minnesota']    # changed St. Paul to Saint Paul

# scraping wikipedia page
# https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
page = ''
while page == '':
    try:
        page = requests.get('https://en.wikipedia.org/wiki/Lists_of_neighborhoods_by_city#United_States')
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue
        
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
items = content.find_all('li')

In [3]:
# STEP 1.1: get directories (what comes after wikipedia base url) for all cities
# creating dict where keys are cities and values are directories
city_directories = {}

# iterate through list of cities on wikipedia page - if they match our cities, get its directory
for item in items:
    wiki_city_cleaned = item.text.strip('*').strip()
    if wiki_city_cleaned == 'Neighborhoods of Chicago':   # hard-coded
        city_directories[cities[0]] = item.a['href']
    for city in cities:
        if wiki_city_cleaned == city.split(',')[0] or wiki_city_cleaned == city: 
            city_directories[city] = item.a['href']

city_directories['Kansas City, Missouri'] = '/wiki/Neighborhoods_of_Kansas_City,_Missouri'   # hard-coded
city_directories

{'Chicago, Illinois': '/wiki/Neighborhoods_of_Chicago',
 'Columbus, Ohio': '/wiki/Neighborhoods_in_Columbus,_Ohio',
 'Detroit, Michigan': '/wiki/Neighborhoods_in_Detroit',
 'Indianapolis, Indiana': '/wiki/Neighborhoods_of_Indianapolis',
 'Milwaukee, Wisconsin': '/wiki/Neighborhoods_of_Milwaukee',
 'Minneapolis, Minnesota': '/wiki/Neighborhoods_of_Minneapolis',
 'Saint Paul, Minnesota': '/wiki/Neighborhoods_of_Saint_Paul',
 'St. Louis, Missouri': '/wiki/Neighborhoods_of_St._Louis',
 'Kansas City, Missouri': '/wiki/Neighborhoods_of_Kansas_City,_Missouri'}

In [4]:
# STEP 1.2: creating functions for scraping wikipedia page & cleaning

base_url = 'https://en.wikipedia.org'

# uses pandas read_html to scrape neighborhoods from table on each wiki page
def read_wiki(city):
    directory = city_directories.get(city)
    wiki = pd.read_html(base_url + directory)
    return wiki

# uses bs4 to scrape neighborhoods based on html tags
def soup(city):
    directory = city_directories.get(city)
    page = requests.get(base_url + directory)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find('div', class_='mw-parser-output')
    return content

# cleans where neighborhoods have multiple names with '/'
# NOTE: still in progress
def cleaned(lst):
    for neighborhood in lst:
        if '/' in neighborhood or ' / ' in neighborhood:
            lst.remove(neighborhood)
            aliases = neighborhood.split('/')
            for item in aliases:
                lst.append(item.strip())
        if '[' in neighborhood:
            lst.remove(neighborhood)
            split = neighborhood.split('[')
            lst.append(split[0].strip())
    return lst

In [7]:
# STEP 1.3: get neighborhoods for chicago, indianapolis, detroit, st. louis
# creating dict where key is city and value is list of neighborhoods
neighborhoods = {}

# chicago neighborhoods
wiki_chicago = read_wiki('Chicago, Illinois')
chicago_neighborhoods = wiki_chicago[0]
chicago_neighborhoods = list(chicago_neighborhoods['Neighborhood'])
neighborhoods['Chicago, Illinois'] = chicago_neighborhoods

# indianapolis neighborhoods
wiki_ind = read_wiki('Indianapolis, Indiana')
ind_neighborhoods = wiki_ind[2]
indianapolis_neighborhoods = list(ind_neighborhoods[ind_neighborhoods.columns[0]])
neighborhoods['Indianapolis, Indiana'] = indianapolis_neighborhoods

# st. louis neighborhoods
wiki_stl = read_wiki('St. Louis, Missouri')
stl_neighborhoods = wiki_stl[1]
stl_neighborhoods = list(stl_neighborhoods['Neighborhood'])
neighborhoods['St. Louis, Missouri'] = stl_neighborhoods

# detroit neighborhoods
wiki_detroit =  read_wiki('Detroit, Michigan')
detroit_neighborhoods = []
for page in wiki_detroit:
    if 'Name' in page.columns:
        detroit_neighborhoods.extend(list(page['Name']))
        
# cleaning detroit neighborhoods
detroit_neighborhoods = cleaned(detroit_neighborhoods)
for n in detroit_neighborhoods:
    if '/' in n:
        detroit_neighborhoods.remove(n)
        names = n.split('/')
        for name in names:
            detroit_neighborhoods.append(name)
neighborhoods['Detroit, Michigan'] = detroit_neighborhoods

In [10]:
# STEP 1.4: get neighborhoods for columbus, milwaukee, kansas city, minneapolis, saint paul 
# NOTE: need fix repetition of code later for efficiency

# columbus neighborhoods
columbus_directory = city_directories.get('Columbus, Ohio')
page = requests.get(base_url + columbus_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
columbus_neighborhoods = [item.text.replace('[edit]', '') for item in content.find_all(['h2', 'h3', 'h4'])][2:-3]
neighborhoods['Columbus, Ohio'] = columbus_neighborhoods

# milwaukee neighborhoods
mlwk_directory = city_directories.get('Milwaukee, Wisconsin')
page = requests.get(base_url + mlwk_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
mlwk_neighborhoods = [item.text.replace('[edit]', '').strip('"') for item in content.find_all('h3')]
neighborhoods['Milwaukee, Wisconsin'] = mlwk_neighborhoods

# kansas neighborhoods
kansas_directory = city_directories.get('Kansas City, Missouri')
page = requests.get(base_url + kansas_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
li_list = [item.text for item in content.find_all('li')]
kansas_neighborhoods = li_list[li_list.index('CBD-Downtown'):li_list.index('Wornall Homestead') + 1]
neighborhoods['Kansas City, Missouri'] = kansas_neighborhoods

# minneapolis neightborhoods
minneapolis_directory = city_directories.get('Minneapolis, Minnesota')
page = requests.get(base_url + minneapolis_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3 = [item.text.replace('[edit]', '').strip('"') for item in content.find_all(['h3'])]
li = [item.text for item in content.find_all(['li']) if item.text[0].isalpha()]
li = li[0:li.index('University') + 1]
b = [item.text for item in content.find_all('b') if item.text[0].isalpha() and item.text not in li]
minneapolis_neighborhoods = h3 + li + b
neighborhoods['Minneapolis, Minnesota'] = minneapolis_neighborhoods

# saint paul neighborhoods
stpaul_directory = city_directories.get('Saint Paul, Minnesota')
page = requests.get(base_url + stpaul_directory)
soup = BeautifulSoup(page.text, 'html.parser')
content = soup.find('div', class_='mw-parser-output')
h3_list = [item.text.strip().split('- ')[1].replace('[edit]', '') for item in content.find_all('h3')]
stpaul_neighborhoods = []
for h3 in h3_list:
    names = h3.split(',')
    for name in names:
        stpaul_neighborhoods.append(name.strip())
neighborhoods['Saint Paul, Minnesota'] = stpaul_neighborhoods

In [18]:
# step 2: use yelp API to get restaurant data for each neighborhood
# starting with detroit
# how many restaurants do we want for each neighborhood? aka what limit

for n in neighborhoods['Detroit, Michigan']:
    try:
        print(yelp_api.search_query(categories='restaurants', location=n + ', Detroit', limit=1))
    except YelpAPI.YelpAPIError as e:
        print('Error')

{'businesses': [{'id': 'quuV34VlUSjFaFWzFDWavw', 'alias': 'jacobys-german-biergarten-detroit', 'name': "Jacoby's German Biergarten", 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/-jeyTIYlGAU5mNd2LWBf0A/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/jacobys-german-biergarten-detroit?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 285, 'categories': [{'alias': 'german', 'title': 'German'}, {'alias': 'pubs', 'title': 'Pubs'}], 'rating': 4.0, 'coordinates': {'latitude': 42.33229, 'longitude': -83.04208}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '624 Brush St', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48226', 'country': 'US', 'state': 'MI', 'display_address': ['624 Brush St', 'Detroit, MI 48226']}, 'phone': '+13139627067', 'display_phone': '(313) 962-7067', 'distance': 245.77177756801947}], 'total': 79, 'region': {'ce

{'businesses': [{'id': 'HYrqw4xlLCNDptBHGrTIbQ', 'alias': 'dime-store-detroit-4', 'name': 'Dime Store', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/tBmxNQQCDvHYj4Fq_drkVw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/dime-store-detroit-4?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 1601, 'categories': [{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}, {'alias': 'newamerican', 'title': 'American (New)'}, {'alias': 'diners', 'title': 'Diners'}], 'rating': 4.5, 'coordinates': {'latitude': 42.3308060633157, 'longitude': -83.0479924085636}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '719 Griswold St', 'address2': 'Ste 180', 'address3': '', 'city': 'Detroit', 'zip_code': '48226', 'country': 'US', 'state': 'MI', 'display_address': ['719 Griswold St', 'Ste 180', 'Detroit, MI 48226']}, 'phone': '+13139629106', 'display_phone': '

{'businesses': [{'id': 'HYrqw4xlLCNDptBHGrTIbQ', 'alias': 'dime-store-detroit-4', 'name': 'Dime Store', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/tBmxNQQCDvHYj4Fq_drkVw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/dime-store-detroit-4?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 1601, 'categories': [{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}, {'alias': 'newamerican', 'title': 'American (New)'}, {'alias': 'diners', 'title': 'Diners'}], 'rating': 4.5, 'coordinates': {'latitude': 42.3308060633157, 'longitude': -83.0479924085636}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '719 Griswold St', 'address2': 'Ste 180', 'address3': '', 'city': 'Detroit', 'zip_code': '48226', 'country': 'US', 'state': 'MI', 'display_address': ['719 Griswold St', 'Ste 180', 'Detroit, MI 48226']}, 'phone': '+13139629106', 'display_phone': '

{'businesses': [{'id': '4p1bXdMdETmyU5qhzFxNxw', 'alias': 'selden-standard-detroit', 'name': 'Selden Standard', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/6THUqNH-HzWDaEiGonIshw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/selden-standard-detroit?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 1160, 'categories': [{'alias': 'newamerican', 'title': 'American (New)'}, {'alias': 'sandwiches', 'title': 'Sandwiches'}], 'rating': 4.5, 'coordinates': {'latitude': 42.34779, 'longitude': -83.06499}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '3921 2nd Ave', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48201', 'country': 'US', 'state': 'MI', 'display_address': ['3921 2nd Ave', 'Detroit, MI 48201']}, 'phone': '+13134385055', 'display_phone': '(313) 438-5055', 'distance': 465.983355696597}], 'total': 624, 'region': {'center':

{'businesses': [{'id': '70zEOA3dsFEwHs5IIFWiaQ', 'alias': 'la-dolce-vita-detroit-2', 'name': 'La Dolce Vita', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/oujdqN_rb4-pojKddUtqVg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/la-dolce-vita-detroit-2?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 393, 'categories': [{'alias': 'italian', 'title': 'Italian'}, {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}], 'rating': 3.5, 'coordinates': {'latitude': 42.420273, 'longitude': -83.106426}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '17546 Woodward Ave', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48203', 'country': 'US', 'state': 'MI', 'display_address': ['17546 Woodward Ave', 'Detroit, MI 48203']}, 'phone': '+13138650331', 'display_phone': '(313) 865-0331', 'distance': 423.2718608262382}], 'total': 209, 'regi

...

{'businesses': [{'id': 'HYrqw4xlLCNDptBHGrTIbQ', 'alias': 'dime-store-detroit-4', 'name': 'Dime Store', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/tBmxNQQCDvHYj4Fq_drkVw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/dime-store-detroit-4?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 1601, 'categories': [{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}, {'alias': 'newamerican', 'title': 'American (New)'}, {'alias': 'diners', 'title': 'Diners'}], 'rating': 4.5, 'coordinates': {'latitude': 42.3308060633157, 'longitude': -83.0479924085636}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '719 Griswold St', 'address2': 'Ste 180', 'address3': '', 'city': 'Detroit', 'zip_code': '48226', 'country': 'US', 'state': 'MI', 'display_address': ['719 Griswold St', 'Ste 180', 'Detroit, MI 48226']}, 'phone': '+13139629106', 'display_phone': '

{'businesses': [{'id': '8ctLeteWujq_EfH2lRSTAw', 'alias': 'supino-pizzeria-detroit', 'name': 'Supino Pizzeria', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/pVTGS-8xlSjRZ65nWyNQdw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/supino-pizzeria-detroit?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 923, 'categories': [{'alias': 'pizza', 'title': 'Pizza'}, {'alias': 'italian', 'title': 'Italian'}], 'rating': 4.5, 'coordinates': {'latitude': 42.345476174914, 'longitude': -83.040064828648}, 'transactions': ['delivery'], 'price': '$$', 'location': {'address1': '2457 Russell St', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48207', 'country': 'US', 'state': 'MI', 'display_address': ['2457 Russell St', 'Detroit, MI 48207']}, 'phone': '+13135677879', 'display_phone': '(313) 567-7879', 'distance': 1007.9396213233653}], 'total': 247, 'region': {'center':

{'businesses': [{'id': 'mURau2Lec6wdp5tEKGFcYg', 'alias': 'beirut-bakery-redford', 'name': 'Beirut Bakery', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/YEHrVey_00PeGX20i3FJrg/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/beirut-bakery-redford?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 149, 'categories': [{'alias': 'bakeries', 'title': 'Bakeries'}, {'alias': 'lebanese', 'title': 'Lebanese'}, {'alias': 'delis', 'title': 'Delis'}], 'rating': 5.0, 'coordinates': {'latitude': 42.38567, 'longitude': -83.29437}, 'transactions': ['delivery', 'pickup'], 'price': '$', 'location': {'address1': '25706 Schoolcraft', 'address2': '', 'address3': '', 'city': 'Redford', 'zip_code': '48239', 'country': 'US', 'state': 'MI', 'display_address': ['25706 Schoolcraft', 'Redford, MI 48239']}, 'phone': '+13135334422', 'display_phone': '(313) 533-4422', 'distance': 852.2568797544209

{'businesses': [{'id': 'Qn3BDpni0V2vtp9P0-85BQ', 'alias': 'taqueria-el-rey-detroit', 'name': 'Taqueria El Rey', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/xwxYA84pXPNe9pt_zJT9vA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/taqueria-el-rey-detroit?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 433, 'categories': [{'alias': 'mexican', 'title': 'Mexican'}, {'alias': 'beerbar', 'title': 'Beer Bar'}, {'alias': 'seafood', 'title': 'Seafood'}], 'rating': 4.5, 'coordinates': {'latitude': 42.3190260345587, 'longitude': -83.0991818010807}, 'transactions': ['delivery'], 'price': '$', 'location': {'address1': '4730 W Vernor Hwy', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48209', 'country': 'US', 'state': 'MI', 'display_address': ['4730 W Vernor Hwy', 'Detroit, MI 48209']}, 'phone': '+13133573094', 'display_phone': '(313) 357-3094', 'distance': 1396

{'businesses': [{'id': 'S4425OWYkTTqcZ8PceVcmg', 'alias': 'capers-detroit', 'name': 'Capers', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/uzJSdsfWoqR7b-WDC6w3-w/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/capers-detroit?adjust_creative=x-ANpASpfZcpKk2SRWabWg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x-ANpASpfZcpKk2SRWabWg', 'review_count': 148, 'categories': [{'alias': 'steak', 'title': 'Steakhouses'}], 'rating': 4.0, 'coordinates': {'latitude': 42.44052, 'longitude': -82.97356}, 'transactions': ['pickup', 'delivery'], 'price': '$$', 'location': {'address1': '14726 Gratiot Ave', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48205', 'country': 'US', 'state': 'MI', 'display_address': ['14726 Gratiot Ave', 'Detroit, MI 48205']}, 'phone': '+13135272100', 'display_phone': '(313) 527-2100', 'distance': 2527.2555762840866}], 'total': 27, 'region': {'center': {'longitude': -82.99870491027832, 'latitude': 42.42732054277333

In [ ]:
# step 3: write results into csv file (or create pandas dataframe?) for each neighborhood